In [24]:
import json
from nltk.corpus import stopwords
import re
import pymorphy2
import gensim.corpora as corpora
import pyLDAvis.gensim_models
import pyLDAvis
import pickle
import os


morph = pymorphy2.MorphAnalyzer()

/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is d

In [7]:
with open("../data/corpus_as_dict_of_norms.json") as f:
    normed_dict = json.load(f)

In [8]:
corpus = [normed_dict[i] for i in normed_dict]

In [10]:
def del_stops(lst):
    for word in lst:
        word = re.sub("[^а-яА-ЯёЁ]"," ", re.sub(r'\((.*?)\)', "", word))
    lst = [word for word in lst if not re.match(r'[a-zA-Z]+', word)]
    lst = [word for word in lst if morph.parse(word)[0].tag.POS not in ['CONJ', 'PRCL', 'INTJ', 'PREP', 'NPRO', 'NUMR']]
    stops = stopwords.words("english") + stopwords.words("russian") + ["это", "который", "наш", "мочь", "год", 
                                            "такой", "знать", "мы", "свой", "один", "другой", "хотеть",
                                            "человек", "всё", "все", "весь", "очень", "думать", "нужно",
                                            "большой", "время", "использовать", "говорить", "сказать",
                                            "иметь", "сделать", "первый", "каждый", "день", "её", "ваш",
                                            "стать", "больший", "ваше", "день", "самый", "понять",
                                            "просто", "ещё", "проблема", "также", "например", "м", "c"]
    return [w for w in lst if w not in stops]

In [11]:
for i in range(len(corpus)):
    corpus[i] = del_stops(corpus[i])

In [12]:
all_words = []
for doc in corpus:
    all_words.extend(doc)

In [13]:
len(all_words)

24000

In [14]:
id2word = corpora.Dictionary(corpus)

In [15]:
docs = corpus
corpus = [id2word.doc2bow(doc) for doc in docs]

In [16]:
print(corpus[0])

[(0, 3), (1, 4), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 4), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 2), (24, 2), (25, 1), (26, 7), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1)]


In [17]:
from gensim.models import LdaMulticore
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=6)

In [18]:
lda_model.print_topics()

[(0,
  '0.006*"электрический" + 0.005*"являться" + 0.005*"город" + 0.003*"поле" + 0.003*"энергия" + 0.003*"судный" + 0.003*"вид" + 0.003*"вода" + 0.003*"часть" + 0.003*"движение"'),
 (1,
  '0.005*"являться" + 0.004*"вид" + 0.003*"называть" + 0.003*"поле" + 0.003*"магнитный" + 0.003*"использоваться" + 0.003*"часть" + 0.003*"электрический" + 0.003*"век" + 0.003*"название"'),
 (2,
  '0.006*"являться" + 0.005*"вода" + 0.004*"море" + 0.004*"часть" + 0.004*"судный" + 0.004*"город" + 0.004*"судно" + 0.003*"океан" + 0.003*"труба" + 0.003*"северный"'),
 (3,
  '0.006*"являться" + 0.005*"судный" + 0.005*"вид" + 0.004*"вода" + 0.004*"город" + 0.004*"земля" + 0.003*"крупный" + 0.003*"судно" + 0.003*"скорость" + 0.003*"северный"'),
 (4,
  '0.007*"судный" + 0.005*"являться" + 0.005*"часть" + 0.003*"корпус" + 0.003*"северный" + 0.003*"груз" + 0.003*"вид" + 0.002*"корабль" + 0.002*"вода" + 0.002*"площадь"'),
 (5,
  '0.006*"являться" + 0.005*"судный" + 0.003*"судно" + 0.003*"использоваться" + 0.003*"еди

In [25]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(LDAvis_prepared, "../data/LDA.html")

In [31]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.037375 -0.020376       1        1  25.640530
2      0.022677  0.041529       2        1  17.005512
4     -0.000275 -0.009083       3        1  16.243647
5      0.014270 -0.032528       4        1  15.658948
0     -0.018022  0.037570       5        1  13.441993
1     -0.056024 -0.017111       6        1  12.009371, topic_info=               Term       Freq      Total Category  logprob  loglift
183   электрический  43.000000  43.000000  Default  30.0000  30.0000
81          энергия  20.000000  20.000000  Default  29.0000  29.0000
1182          труба  24.000000  24.000000  Default  28.0000  28.0000
618            поле  42.000000  42.000000  Default  27.0000  27.0000
2509            днк  12.000000  12.000000  Default  26.0000  26.0000
...             ...        ...        ...      ...      ...      ...
361       население   5.149597  45.026753   Topic6  -6.3274  -0.0489
58        различный   4.733545  36.140874   Topic6  -6.4117   0.0867
539     государство   4.467575  27.025149   Topic6  -6.4695   0.3196
472    представлять   4.391925  26.214472   Topic6  -6.4866   0.3329
156          обычно   4.380134  28.604154   Topic6  -6.4892   0.2430

[485 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
3621      5  0.449766         агрегат
1547      4  0.897062  адмиралтейство
3886      5  0.666564           адрес
2494      2  0.629061       азотистый
4653      5  0.463638        алкалоид
...     ...       ...             ...
6686      1  0.234227          япония
6686      2  0.156152          япония
6686      4  0.078076          япония
6686      5  0.078076          япония
6686      6  0.468455          япония

[1145 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 6, 1, 2])

In [32]:
LDAvis_prepared.topic_order

[4, 3, 5, 6, 1, 2]